In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVC
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error

# 小さなデータセットの用意
以前も利用した回帰のデータセットを用意します。


House Prices: Advanced Regression Techniques


この中のtrain.csvをダウンロードし、目的変数としてSalePrice、説明変数として、GrLivAreaとYearBuiltを使います。


train.csvを学習用（train）8割、検証用（val）2割に分割してください。

In [2]:
df = pd.read_csv("/Users/takahashihideyuki/dive/diveintocode-ml/Week4/train.csv", encoding="utf-8")
df

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


In [3]:
# GrLivArea: Above grade (ground) living area square feet（グレード（地上）のリビングエリアの平方フィート）
# YearBuilt: Original construction date（元の建設日）
X = df[["GrLivArea", "YearBuilt"]] #データを抜き出し
X

,GrLivArea,YearBuilt
0,1710,2003
1,1262,1976
2,1786,2001
3,1717,1915
4,2198,2000
...,...,...
1455,1647,1999
1456,2073,1978
1457,2340,1941
1458,1078,1950


In [4]:
# SalePrice: the property's sale price in dollars. This is the target variable that you're trying to predict.（資産のドルでの販売価格。 これは、予測しようとしているターゲット変数です。）
y = df["SalePrice"] #目的変数を抜き出し
y

0       208500
1       181500
2       223500
3       140000
4       250000
         ...  
1455    175000
1456    210000
1457    266500
1458    142125
1459    147500
Name: SalePrice, Length: 1460, dtype: int64

In [5]:
# 訓練データと検証データの分割。訓練データ80%、検証データ20%として分割する。
X = np.array(X)
y = np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=123)
display(X_train.shape)
display(X_test.shape)
display(y_train.shape)
display(y_test.shape)

(1168, 2)

(292, 2)

(1168,)

(292,)

In [6]:
#標準化
scaler = StandardScaler()
scaler.fit(X_train) #訓練用のデータでfit
X_train_std = scaler.transform(X_train) #訓練用データをtransform
X_train_std

array([[ 2.09562928,  0.72737914],
       [-0.17116889, -0.35870212],
       [ 2.51657947,  0.20079429],
       ...,
       [ 1.18294965, -0.09540969],
       [-1.40235508,  0.03623652],
       [-0.56417991, -0.98402163]])

In [7]:
X_test_std = scaler.transform(X_test) #検証用データをtransform
X_test_std

array([[ 9.57573663e-01,  9.90671570e-01],
       [ 4.71432068e-01, -2.33339532e+00],
       [ 3.48499710e-01,  1.08940623e+00],
       [ 1.07305557e+00,  8.91936910e-01],
       [ 6.91079888e-02, -8.52375419e-01],
       [ 1.69548675e-02,  1.08940623e+00],
       [ 1.73428265e+00,  1.08940623e+00],
       [-1.00748144e+00,  1.34971182e-01],
       [-7.31814946e-01, -2.59967458e-01],
       [-7.46715838e-01,  1.18814089e+00],
       [-7.57891507e-01,  1.08940623e+00],
       [ 7.34060286e-01,  6.94467590e-01],
       [-8.36121189e-01,  1.12231778e+00],
       [-1.06708501e+00, -9.54096916e-02],
       [ 1.54988411e+00,  2.99528949e-01],
       [ 1.47337671e-01, -4.57436779e-01],
       [ 4.65844233e-01, -3.58702119e-01],
       [ 1.19226271e+00,  1.18814089e+00],
       [ 4.60256399e-01,  1.18814089e+00],
       [ 1.67826397e-01, -2.27055905e-01],
       [-8.69648195e-01, -6.24981382e-02],
       [ 2.66544805e-01,  1.67882735e-01],
       [-8.95724756e-01, -6.87817652e-01],
       [-7.

# 【問題1】ブレンディングのスクラッチ実装
ブレンディング をスクラッチ実装し、単一モデルより精度があがる例を 最低3つ 示してください。精度があがるとは、検証用データに対する平均二乗誤差（MSE）が小さくなることを指します。


### ブレンディングとは
ブレンディングとは、N個の多様なモデルを独立して学習させ、推定結果を重み付けした上で足し合わせる方法です。最も単純には平均をとります。多様なモデルとは、以下のような条件を変化させることで作り出すものです。


- 手法（例：線形回帰、SVM、決定木、ニューラルネットワークなど）
- ハイパーパラメータ（例：SVMのカーネルの種類、重みの初期値など）
- 入力データの前処理の仕方（例：標準化、対数変換、PCAなど）

重要なのはそれぞれのモデルが大きく異なることです。


回帰問題でのブレンディングは非常に単純であるため、scikit-learnには用意されていません。


《補足》


分類問題の場合は、多数決を行います。回帰問題に比べると複雑なため、scikit-learnにはVotingClassifierが用意されています。


sklearn.ensemble.VotingClassifier — scikit-learn 0.21.3 documentation

## 比較ベースとする単一モデル

In [8]:
# 線形回帰　
lin = LinearRegression()
lin.fit(X_train_std, y_train) # 学習
pred = lin.predict(X_test_std) #クラスの予測

#検証データと予測結果の平均二乗誤差
print("平均二乗誤差（MSE）：{:,.2f}".format(mean_squared_error(y_test, pred)))

平均二乗誤差（MSE）：1,907,504,023.48


In [9]:
# 決定木
dtr = DecisionTreeRegressor()
dtr.fit(X_train_std, y_train) # 学習
pred = dtr.predict(X_test_std) #クラスの予測

#検証データと予測結果の平均二乗誤差
print("平均二乗誤差（MSE）：{:,.2f}".format(mean_squared_error(y_test, pred)))

平均二乗誤差（MSE）：2,586,129,401.82


In [10]:
# SVM
svr = SVR()
svr.fit(X_train_std, y_train) # 学習
pred = svr.predict(X_test_std) #クラスの予測

#検証データと予測結果の平均二乗誤差
print("平均二乗誤差（MSE）：{:,.2f}".format(mean_squared_error(y_test, pred)))

平均二乗誤差（MSE）：6,554,411,131.86


## ブレンディング

In [11]:
class Blending():
    
    def __init__(self, models):
        self.models = models #モデルをインスタンス変数化
        
    def fit(self, X, y):
        model_fitted_list = [] #学習したモデルを格納するリスト
        for model in self.models: #各モデルで学習を実施
            model_fitted = model.fit(X, y) # 学習
            model_fitted_list.append(model_fitted) #学習したモデルをリストへ格納
  
        self.model_fitted_list = model_fitted_list #学習したモデルを格納したリストをインスタンス変数化
            
    def predict(self, X):
        pred_list = [] #推定した値を格納するリスト
        for model_fitted in self.model_fitted_list: #各モデルで推定を実施
            pred = model_fitted.predict(X) #推定
            pred_list.append(pred) #推定した値をリストへ格納
    
        pred_blend = np.mean(np.array(pred_list), axis=0) #各モデルでの推定値を平均
        
        return pred_blend

### No.1　線形回帰に決定木をブレンディング

In [12]:
bld = Blending(models=[LinearRegression(), 
                                          DecisionTreeRegressor()]
                                                                                      )
                                         
bld.fit(X_train_std, y_train)
pred_blend = bld.predict(X_test_std)
print("平均二乗誤差（MSE、全モデルの平均）：{:,.2f}".format(mean_squared_error(y_test, pred_blend)))

平均二乗誤差（MSE、全モデルの平均）：1,800,339,538.86


### No.2　線形回帰にSVRをブレンディング

In [13]:
bld = Blending(models=[LinearRegression(),
                                          SVR()]
                                                                            )
                                         
bld.fit(X_train_std, y_train)
pred_blend = bld.predict(X_test_std)
print("平均二乗誤差（MSE、全モデルの平均）：{:,.2f}".format(mean_squared_error(y_test, pred_blend)))

平均二乗誤差（MSE、全モデルの平均）：3,233,722,326.27


### No.3　決定木にSVRをブレンディング

In [14]:
bld = Blending(models=[DecisionTreeRegressor(),
                                          SVR()]
                                                                                     )
                                         
bld.fit(X_train_std, y_train)
pred_blend = bld.predict(X_test_std)
print("平均二乗誤差（MSE、全モデルの平均）：{:,.2f}".format(mean_squared_error(y_test, pred_blend)))

平均二乗誤差（MSE、全モデルの平均）：2,820,214,288.81


### No.4　線形回帰に決定木とSVRをブレンディング

In [15]:
bld = Blending(models=[LinearRegression(), 
                                          DecisionTreeRegressor(), 
                                          SVR()]
                                                                                      )
                                         
bld.fit(X_train_std, y_train)
pred_blend = bld.predict(X_test_std)
print("平均二乗誤差（MSE、全モデルの平均）：{:,.2f}".format(mean_squared_error(y_test, pred_blend)))

平均二乗誤差（MSE、全モデルの平均）：2,354,158,637.24


### No.5　No.1より決定木のmax_depthを変更

In [16]:
for i in range(1, 21):
    bld = Blending(models=[LinearRegression(),
                                              DecisionTreeRegressor(max_depth = i)]
                                                                                                                   )

    bld.fit(X_train_std, y_train)
    pred_blend = bld.predict(X_test_std)
    print("平均二乗誤差（MSE、全モデルの平均）、max_depth = {}：{:,.2f}".format(i, mean_squared_error(y_test, pred_blend)))

平均二乗誤差（MSE、全モデルの平均）、max_depth = 1：2,677,431,180.02
平均二乗誤差（MSE、全モデルの平均）、max_depth = 2：2,146,855,941.35
平均二乗誤差（MSE、全モデルの平均）、max_depth = 3：1,890,238,393.21
平均二乗誤差（MSE、全モデルの平均）、max_depth = 4：1,728,066,982.25
平均二乗誤差（MSE、全モデルの平均）、max_depth = 5：1,605,807,203.26
平均二乗誤差（MSE、全モデルの平均）、max_depth = 6：1,497,929,407.96
平均二乗誤差（MSE、全モデルの平均）、max_depth = 7：1,560,915,070.11
平均二乗誤差（MSE、全モデルの平均）、max_depth = 8：1,621,186,714.63
平均二乗誤差（MSE、全モデルの平均）、max_depth = 9：1,731,995,703.64
平均二乗誤差（MSE、全モデルの平均）、max_depth = 10：1,665,754,259.95
平均二乗誤差（MSE、全モデルの平均）、max_depth = 11：1,691,091,447.25
平均二乗誤差（MSE、全モデルの平均）、max_depth = 12：1,689,687,318.97
平均二乗誤差（MSE、全モデルの平均）、max_depth = 13：1,730,436,471.06
平均二乗誤差（MSE、全モデルの平均）、max_depth = 14：1,766,487,048.66
平均二乗誤差（MSE、全モデルの平均）、max_depth = 15：1,735,369,858.73
平均二乗誤差（MSE、全モデルの平均）、max_depth = 16：1,834,734,181.35
平均二乗誤差（MSE、全モデルの平均）、max_depth = 17：1,735,757,582.01
平均二乗誤差（MSE、全モデルの平均）、max_depth = 18：1,743,515,105.65
平均二乗誤差（MSE、全モデルの平均）、max_depth = 19：1,815,386,813.08
平均二乗誤差（MSE、全モデルの平均）、m

### 設定値次第で結果が向上する。以降はmax_depth = 6とする。

### No.6　No.2よりSVRのkernelを変更

In [17]:
for kernel in ['linear', 'poly',  'rbf',  'sigmoid']:
    bld = Blending(models=[LinearRegression(), 
                                              SVR(kernel=kernel)]
                                                                                 )

    bld.fit(X_train_std, y_train)
    pred_blend = bld.predict(X_test_std)
    print("平均二乗誤差（MSE、全モデルの平均）、kernel={}：{:,.2f}".format(kernel, mean_squared_error(y_test, pred_blend)))

平均二乗誤差（MSE、全モデルの平均）、kernel=linear：3,205,825,581.65
平均二乗誤差（MSE、全モデルの平均）、kernel=poly：3,199,556,520.39
平均二乗誤差（MSE、全モデルの平均）、kernel=rbf：3,233,722,326.27
平均二乗誤差（MSE、全モデルの平均）、kernel=sigmoid：3,227,656,631.72


### 設定値次第で結果が向上する。以降はkernel = "poly"とする。

### No.7　No.6よりSVRのCを変更

In [18]:
for i in np.arange(1, 1000, 100):
    bld = Blending(models=[LinearRegression(), 
                                              SVR(kernel="poly", C=i)]
                                                                                 )

    bld.fit(X_train_std, y_train)
    pred_blend = bld.predict(X_test_std)
    print("平均二乗誤差（MSE、全モデルの平均）、C = {}：{:,.2f}".format(i, mean_squared_error(y_test, pred_blend)))

平均二乗誤差（MSE、全モデルの平均）、C = 1：3,199,556,520.39
平均二乗誤差（MSE、全モデルの平均）、C = 101：2,198,110,329.76
平均二乗誤差（MSE、全モデルの平均）、C = 201：2,115,172,620.95
平均二乗誤差（MSE、全モデルの平均）、C = 301：2,080,423,154.37
平均二乗誤差（MSE、全モデルの平均）、C = 401：2,060,464,036.90
平均二乗誤差（MSE、全モデルの平均）、C = 501：2,045,922,021.74
平均二乗誤差（MSE、全モデルの平均）、C = 601：2,027,533,787.06
平均二乗誤差（MSE、全モデルの平均）、C = 701：2,018,921,365.82
平均二乗誤差（MSE、全モデルの平均）、C = 801：2,011,484,039.69
平均二乗誤差（MSE、全モデルの平均）、C = 901：1,998,713,555.59


### 値をあげると結果が向上するが、過剰適合になる。以降はC = 500とする。

### No.8　No.7よりSVRのcoef0を変更

In [19]:
for i in np.arange(0, 1, 0.1):
    bld = Blending(models=[LinearRegression(), 
                                              SVR(kernel="poly", C=500, coef0 = i)]
                                                                                 )

    bld.fit(X_train_std, y_train)
    pred_blend = bld.predict(X_test_std)
    print("平均二乗誤差（MSE、全モデルの平均）、C = {}：{:,.2f}".format(i, mean_squared_error(y_test, pred_blend)))

平均二乗誤差（MSE、全モデルの平均）、C = 0.0：2,045,922,022.95
平均二乗誤差（MSE、全モデルの平均）、C = 0.1：1,855,003,394.02
平均二乗誤差（MSE、全モデルの平均）、C = 0.2：1,757,809,106.27
平均二乗誤差（MSE、全モデルの平均）、C = 0.30000000000000004：1,686,194,604.07
平均二乗誤差（MSE、全モデルの平均）、C = 0.4：1,710,196,419.81
平均二乗誤差（MSE、全モデルの平均）、C = 0.5：1,709,850,275.14
平均二乗誤差（MSE、全モデルの平均）、C = 0.6000000000000001：1,709,689,919.20
平均二乗誤差（MSE、全モデルの平均）、C = 0.7000000000000001：1,744,169,316.58
平均二乗誤差（MSE、全モデルの平均）、C = 0.8：1,775,642,247.64
平均二乗誤差（MSE、全モデルの平均）、C = 0.9：1,805,095,170.08


### 設定値次第で結果が向上する。以降はcoef0 = 0.3とする。

### No.9　線形回帰に決定木（max_depth = 6）をブレンディング

In [20]:
bld = Blending(models=[DecisionTreeRegressor(max_depth=6),
                                           LinearRegression()]
                                                                                     )
                                         
bld.fit(X_train_std, y_train)
pred_blend = bld.predict(X_test_std)
print("平均二乗誤差（MSE、全モデルの平均）：{:,.2f}".format(mean_squared_error(y_test, pred_blend)))

平均二乗誤差（MSE、全モデルの平均）：1,497,929,407.96


### No.10　No.8にSVRのkernel="poly", C=500, coef0 = 0.3をブレンディング

In [21]:
bld = Blending(models=[LinearRegression(),
                                         DecisionTreeRegressor(max_depth = 6),
                                         SVR(kernel="poly", C=500, coef0 = 0.3)]
                                                                                                                )
                                         
bld.fit(X_train_std, y_train)
pred_blend = bld.predict(X_test_std)
print("平均二乗誤差（MSE、全モデルの平均）：{:,.2f}".format(mean_squared_error(y_test, pred_blend)))

平均二乗誤差（MSE、全モデルの平均）：1,475,784,934.11


# 平均二乗誤差まとめ
#### 《単一モデル》<br>
線形回帰：1,907,504,023.48<br>
決定木：　2,586,129,401.82<br>
SVR:       6,554,411,131.86<br>

#### 《精度向上ブレンディング》<br>
No.1　線形回帰に決定木をブレンディング：　1,800,339,538.86<br>
No.5　No.1より決定木のmax_depthを変更：　1,497,929,407.96<br>
No.8　線形回帰にSVRのkernel, C, coef0を変更したものをブレンディング： 1,686,194,604.07<br>
No.10　No.1,5,8をブレンディング: 1,475,784,934.11<br>

# 【問題2】バギングのスクラッチ実装
バギング をスクラッチ実装し、単一モデルより精度があがる例を 最低1つ 示してください。


### バギングとは
バギングは入力データの選び方を多様化する方法です。学習データから重複を許した上でランダムに抜き出すことで、N種類のサブセット（ ブートストラップサンプル ）を作り出します。それらによってモデルをN個学習し、推定結果の平均をとります。ブレンディングと異なり、それぞれの重み付けを変えることはありません。


sklearn.model_selection.train_test_split — scikit-learn 0.21.3 documentation


scikit-learnのtrain_test_splitを、shuffleパラメータをTrueにして使うことで、ランダムにデータを分割することができます。これによりブートストラップサンプルが手に入ります。


推定結果の平均をとる部分はブースティングと同様の実装になります。

## バギング

In [22]:
from copy import deepcopy
class Bagging():
    
    def __init__(self, model, n_set):
        self.model = model #モデルをインスタンス変数化
        self.n_set = n_set #サンプル数をインスタンス変数化
        
    def fit(self, X, y):
        model_fitted_list = [] #学習したモデルを格納するリスト
        for i in range(self.n_set):
            X_train, y_train = self.split(X, y) #split関数にてブートストラップサンプル を作成 
            model_fitted = deepcopy(self.model).fit(X_train, y_train) #学習
            model_fitted_list.append(model_fitted) #学習したモデルをリストへ格納

        self.model_fitted_list = model_fitted_list #学習したモデルを格納したリストをインスタンス変数化

    #データを分割
    def split(self, X, y):
        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.5)
        return X_train, y_train
            
    def pred(self, X):
        pred_list = [] #推定した値を格納するリスト
        for model_fitted in (self.model_fitted_list):
            pred = model_fitted.predict(X) #推定
            pred_list.append(pred) #推定した値をリストへ格納

        pred_bagg = np.mean(np.array(pred_list), axis=0) #各モデルでの推定値を平均
              
        return pred_bagg

In [23]:
#決定木
bag = Bagging(model=DecisionTreeRegressor(), n_set=10)
bag.fit(X_train_std, y_train)
pred_bagg = bag.pred(X_test_std)
print("平均二乗誤差（MSE、全モデルの平均）：{:,.2f}".format(mean_squared_error(y_test, pred_bagg)))

平均二乗誤差（MSE、全モデルの平均）：1,655,081,724.84


In [24]:
#線形回帰
bag = Bagging(LinearRegression(), n_set=10)
bag.fit(X_train_std, y_train)
pred_bagg = bag.pred(X_test_std)
print("平均二乗誤差（MSE、全モデルの平均）：{:,.2f}".format(mean_squared_error(y_test, pred_bagg)))

平均二乗誤差（MSE、全モデルの平均）：1,886,349,747.29


In [25]:
#SVM
bag = Bagging(SVR(), n_set=10)
bag.fit(X_train_std, y_train)
pred_bagg = bag.pred(X_test_std)
print("平均二乗誤差（MSE、全モデルの平均）：{:,.2f}".format(mean_squared_error(y_test, pred_bagg)))

平均二乗誤差（MSE、全モデルの平均）：6,566,628,066.90


## 決定木にて精度向上を確認。線形回帰及びSVMでは大きな変化は無し。

# 【問題3】スタッキングのスクラッチ実装
スタッキング をスクラッチ実装し、単一モデルより精度があがる例を 最低1つ 示してください。


### スタッキングとは
スタッキングの手順は以下の通りです。最低限ステージ0とステージ1があればスタッキングは成立するため、それを実装してください。まずは $K_0=3, M_0=2$ 程度にします。


《学習時》


（ステージ $0$ ）


- 学習データを $K_0$ 個に分割する。
- 分割した内の $(K_0 - 1)$ 個をまとめて学習用データ、残り $1$ 個を推定用データとする組み合わせが $K_0$ 個作れる。
- あるモデルのインスタンスを $K_0$ 個用意し、異なる学習用データを使い学習する。
- それぞれの学習済みモデルに対して、使っていない残り $1$ 個の推定用データを入力し、推定値を得る。（これをブレンドデータと呼ぶ）
- さらに、異なるモデルのインスタンスも $K_0$ 個用意し、同様のことを行う。モデルが $M_0$ 個あれば、 $M_0$ 個のブレンドデータが得られる。

（ステージ $n$ ）


- ステージ $n-1$ のブレンドデータを$M_{n-1}$ 次元の特徴量を持つ学習用データと考え、 $K_n$ 個に分割する。以下同様である。

（ステージ $N$ ）＊最後のステージ


- ステージ $N-1$ の $M_{N-1}$ 個のブレンドデータを$M_{N-1}$ 次元の特徴量の入力として、1種類のモデルの学習を行う。これが最終的な推定を行うモデルとなる。

《推定時》


（ステージ $0$ ）


- テストデータを $K_0×M_0$ 個の学習済みモデルに入力し、$K_0×M_0$ 個の推定値を得る。これを $K_0$ の軸で平均値を求め $M_0$ 次元の特徴量を持つデータを得る。（ブレンドテストと呼ぶ）

（ステージ $n$ ）


- ステージ $n-1$ で得たブレンドテストを $K_n×M_n$ 個の学習済みモデルに入力し、$K_n×M_n$ 個の推定値を得る。これを $K_n$ の軸で平均値を求め $M_0$ 次元の特徴量を持つデータを得る。（ブレンドテストと呼ぶ）

（ステージ $N$ ）＊最後のステージ


- ステージ $N-1$ で得たブレンドテストを学習済みモデルに入力し、推定値を得る。

## スタッキング

In [26]:
class ScratchStackingRegressor():
    """
    ステージ0と最終ステージのみ
    """ 
    def __init__(self, n_splits, models, final_model):
        self.n_splits = n_splits
        self.models = models
        self.final_model = final_model
        
    def fit(self, X, y):
        model_fitted_list = [] #学習したモデルを格納するリスト
        pred_list = [] #ブレンドデータ の推定値を格納するリスト
        y_val_list = [] #y_valを格納するリスト
        
        #分割したトレインデータにて、各モデルの学習と推定を実施
        kf = KFold(n_splits=self.n_splits, random_state=0, shuffle=True) #トレインデータを分割
        for train_index, val_index in kf.split(X):
            X_train_kf, X_val = X[train_index], X[val_index]
            y_train_kf, y_val = y[train_index], y[val_index]
            for model in deepcopy(self.models):
                model_fitted = model.fit(X_train_kf, y_train_kf) # 学習
                model_fitted_list.append(model_fitted) #学習したモデルをリストへ格納

                pred = model_fitted.predict(X_val) #ブレンドデータの推定値                 
                pred_list.append(pred) #ブレンドデータの推定値をリストに格納
                
            y_val_list.extend(y_val) #y_valをリストへ格納
        
        blend_data = np.vstack((np.concatenate((pred_list[i::len(self.models)])) for i in range(len(self.models)))).T #ステージ０のブレンドデータ
        
        self.final_model_fitted = self.final_model.fit(blend_data, np.array(y_val_list).reshape(-1, 1)) #最終モデル
        
        self.model_fitted_list = model_fitted_list #学習したモデルを格納したリストをインスタンス変数化
        

    def predict(self, X):
        pred_list = [] #推定した値を格納するリスト
        for model_fitted in self.model_fitted_list:
            pred = model_fitted.predict(X) #推定
            pred_list.append(pred) #推定した値をリストへ格納
        
        blend_test = np.vstack(np.mean(np.array(pred_list[i::len(self.models)]), axis=0) for i in range(len(self.models))).T #ステージ０のブレンドテスト 
        
        last_pred = self.final_model_fitted.predict(blend_test) #最後のステージの推定値
        
        return last_pred

In [27]:
ssr = ScratchStackingRegressor(n_splits=3, models=[LinearRegression(), DecisionTreeRegressor()], final_model=LinearRegression())
ssr.fit(X_train_std, y_train)
last_pred = ssr.predict(X_test_std)
print("平均二乗誤差（MSE）：{:,.2f}".format(mean_squared_error(y_test, last_pred))) #mse算出

平均二乗誤差（MSE）：1,683,167,511.06


/Users/takahashihideyuki/.pyenv/versions/anaconda3-2019.10/lib/python3.7/site-packages/ipykernel_launcher.py:29: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
/Users/takahashihideyuki/.pyenv/versions/anaconda3-2019.10/lib/python3.7/site-packages/ipykernel_launcher.py:42: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.


In [28]:
from sklearn.ensemble import StackingRegressor

In [29]:
sr = StackingRegressor(estimators=[('lr', LinearRegression()), ("dtr", DecisionTreeRegressor())], final_estimator=LinearRegression(), cv=3)
sr.fit(X_train_std, y_train)
pred = sr.predict(X_test_std)
print("平均二乗誤差（MSE）：{:,.2f}".format(mean_squared_error(y_test, pred))) #mse算出

平均二乗誤差（MSE）：1,760,988,753.50


## sklearnと同等の結果になっていることを確認。